## Import

In [1]:
from tqdm import tqdm

import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import xlwings as xw
import investpy
import datetime
import os, re

## Load data

In [2]:
data_path = os.getcwd() + "\\datasets\\"

In [3]:
def read_xlsx(name):
    instance = xw.App(visible = False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def read_data(name):
    try :
        output = pd.read_excel(name)
    except ValueError:    
        output = read_xlsx(name)
    return output

In [4]:
def get_stock_list():
    if os.path.isfile(data_path + "df_krx.xlsx") & os.path.isfile(data_path + "df_america.xlsx"):
        df_krx = read_data(data_path + "df_krx.xlsx")
        df_america = read_data(data_path + "df_america.xlsx")
        
    else:
        df_krx = fdr.StockListing('KRX')
        df_sp500 = fdr.StockListing('S&P500')
        df_nas = fdr.StockListing('NASDAQ')
        df_nyse = fdr.StockListing('NYSE')
        df_amex = fdr.StockListing('AMEX')

        df_america = pd.concat([df_nas, df_nyse, df_amex]).drop_duplicates()
        append_code = [x for x in df_sp500.Symbol if x not in list(df_america.Symbol)]
        append_df = df_sp500.loc[df_sp500.Symbol.isin(append_code)]
        del append_df['Sector']
        df_america = pd.concat([df_america, append_df])
        df_america.index = [x for x in range(df_america.shape[0])]
        
        df_krx.to_excel(data_path + "df_krx.xlsx", index = False)
        df_america.to_excel(data_path + "df_america.xlsx", index = False)
    
    return df_krx, df_america

In [5]:
# data 불러오기
if os.path.isfile(data_path + "df_infos.xlsx"):
    df_infos = read_data(data_path + "df_infos.xlsx")
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
else:
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
    df_infos = pd.concat([df_krx, df_america])
    df_infos = df_infos.dropna()
    df_infos.index = [x for x in range(df_infos.shape[0])]
    
    symbol_list = []

    for symbol, name in zip(df_infos.Symbol, df_infos.Name):
        if " PR " in symbol:
            symbol_list.append(re.sub(" PR ","/",symbol))
        elif (".U" in symbol) and ("Units" in name):
            symbol_list.append(re.sub(".U","/UN",symbol))
        elif "\." in symbol:
            symbol_list.append(re.sub("\.", "/", symbol))
        elif " RT" in symbol:
            symbol_list.append(symbol.split(" ")[0])
        elif symbol == 'BRKB':
            symbol_list.append("BRK/B")
        elif symbol == 'BFB':
            symbol_list.append("BF/B")
        else:
            symbol_list.append(symbol)

    df_infos['WINK_symbols'] = symbol_list
    df_infos.to_excel(data_path + "df_infos.xlsx", index = False)

* 환율 갱신

In [6]:
# data 불러오기
if os.path.isfile(data_path + "currency.xlsx"):
    currency = read_data(data_path + "currency.xlsx")

start_date = np.max(currency.Date)
try :
    append_curr = fdr.DataReader("USD/KRW", start = start_date)
    append_curr = append_curr.reset_index()

    currency = currency.loc[currency.Date < start_date, :]
    currency = pd.concat([currency, append_curr])
    currency.index = [x for x in range(currency.shape[0])]
    currency.to_excel(data_path + "currency.xlsx", index = False)
    
except:
    print("추가행 없음")
    pass

* datasets에 데이터 있는지 확인

* 있으면 append

* 없으면 전체 수집

* 수집할 때 원화는 그대로 수집
* 외화는 환율 곱해서 원화로 변경

In [7]:
if os.path.isfile(data_path + "total_df.xlsx"):
    stock_df = read_data(data_path + "total_df.xlsx")
    start_date = np.max(stock_df.Date)
    
    # 추가될 데이터
    origin_df = stock_df.loc[stock_df.Date < start_date, :]
    origin_df.tail()
    
else:
    print("기존 데이터 없음")
    start_date = None

기존 데이터 없음


In [8]:
# symbols
symbol_list = list(df_infos.Symbol.unique())

In [9]:
def change_name(symbol, df, opt = True):
    if opt:
        output = df.loc[df.Symbol == symbol, 'Name'].values[0]
    else:
        output = df.loc[df.Symbol == symbol, 'WINK_symbols'].values[0]
    return output

def make_df(name, df):
    temp_df = pd.DataFrame(df.Close)
    temp_df.columns = ['{}'.format(name)]
    return temp_df.reset_index()

* 국내는 이름
* 해외는 심볼

In [26]:
start_idx = 7815
save_cnt = 35
check_idx = 0
option = True

In [27]:
if not os.path.isdir(data_path + "parts"):
    os.mkdir(data_path + "parts")
part_path = data_path + "parts\\"

In [28]:
try:
    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        try:
            df = fdr.DataReader(symbol, start = start_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        if start_idx > df_krx.shape[0]:
            option = False    

        name = change_name(symbol, df_infos, option)

        if check_idx == 0 :
            stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

        check_idx += 1
        start_idx += 1

        if (check_idx % 100 == 0) and (check_idx > 10):
            stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)
            del stock_df

            check_idx = 0
            save_cnt += 1
            time.sleep(10)

    stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)

except:
    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        try:
            df = fdr.DataReader(symbol, start = start_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        if start_idx > df_krx.shape[0]:
            option = False    

        name = change_name(symbol, df_infos, option)

        if check_idx == 0 :
            stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

        check_idx += 1
        start_idx += 1

        if (check_idx % 100 == 0) and (check_idx > 10):
            stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)
            del stock_df

            check_idx = 0
            save_cnt += 1
            time.sleep(10)

    stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)

  1%|▋                                                                               | 74/8381 [00:07<10:06, 13.70it/s]C:\Windows\python_windows\python_base_env\python-3.8.9.amd64\lib\site-packages\FinanceDataReader\data.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(more)
C:\Windows\python_windows\python_base_env\python-3.8.9.amd64\lib\site-packages\FinanceDataReader\data.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(more)
  1%|▋                                                                             | 76/8381 [00:12<1:51:23,  1.24it/s]C:\Windows\python_windows\python_base_env\python-3.8.9.amd64\lib\site-packages\FinanceDataReader\data.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

HTTP 429 error. Retry after 86400 seconds (1 day, 0:00:00)


  0%|▏                                                                             | 19/8280 [00:59<4:15:28,  1.86s/it]

HTTP 429 error. Retry after 86400 seconds (1 day, 0:00:00)


  0%|▏                                                                             | 20/8280 [01:00<3:10:10,  1.38s/it]

...

HTTP 429 error. Retry after 85562 seconds (23:46:02)


 25%|██████████████████▉                                                         | 2063/8280 [14:58<1:33:43,  1.11it/s]

HTTP 429 error. Retry after 85561 seconds (23:46:01)


 25%|███████████████████▍                                                          | 2063/8280 [15:13<45:53,  2.26it/s]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [29]:
stock_df

,Date,SBUX,SNY,ADP,JD,MDLZ,GILD,REGN,PDD,ADI,VRTX,AMAT,ISRG,BKNG,CME,ABNB,FISV,CSX
0,1992-06-29,0.36,NaN,8.47,NaN,NaN,0.32,11.12,NaN,1.63,3.62,0.62,NaN,NaN,NaN,NaN,1.09,1.75
1,1992-06-30,0.35,NaN,8.42,NaN,NaN,0.38,11.25,NaN,1.60,3.75,0.62,NaN,NaN,NaN,NaN,1.09,1.76
2,1992-07-01,0.36,NaN,8.47,NaN,NaN,0.38,11.00,NaN,1.63,3.75,0.67,NaN,NaN,NaN,NaN,1.17,1.83
3,1992-07-02,0.36,NaN,8.42,NaN,NaN,0.38,10.88,NaN,1.63,3.50,0.59,NaN,NaN,NaN,NaN,1.17,1.82
4,1992-07-06,0.36,NaN,8.30,NaN,NaN,0.37,10.50,NaN,1.60,3.75,0.59,NaN,NaN,NaN,NaN,1.17,1.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10724,1992-06-22,NaN,NaN,8.18,NaN,NaN,0.31,10.00,NaN,1.58,3.56,0.57,NaN,NaN,NaN,NaN,1.10,1.66
10725,1992-06-23,NaN,NaN,8.32,NaN,NaN,0.32,10.75,NaN,1.54,3.50,0.54,NaN,NaN,NaN,NaN,1.12,1.65
10726,1992-06-24,NaN,NaN,8.27,NaN,NaN,0.31,10.75,NaN,1.56,3.56,0.56,NaN,NaN,NaN,NaN,1.10,1.69
10727,1992-06-25,NaN,NaN,8.22,NaN,NaN,0.34,10.75,NaN,1.58,3.56,0.57,NaN,NaN,NaN,NaN,1.12,1.68


In [ ]:
stock_df.head()

* 예외 케이스

In [86]:
'''
except_symbol = []
except_name = []

for idx, symbol in enumerate(symbol_list[2091]):
    try:
        df = fdr.DataReader(symbol)
        
    except ValueError:
        name = change_name(symbol, df_krx)
        except_symbol.append(symbol)
        except_name.append(name)
        
    if df.shape[0] == 0:
        except_symbol.append(symbol)
        except_name.append(name)

np.save("except_symbol.npy", except_symbol)
np.save("except_name.npy", except_name)
'''